# 调用GPT4接口获取变化计数任务的答案

In [10]:
import os, json, re, ast, base64
from openai import OpenAI
import concurrent.futures
import threading
from time import sleep
# set the key
API_SECRET_KEY = "sk-zk2cb5c3fd555a52291f3fb44ea76c226ff66bad83da6099"
BASE_URL = "https://flag.smarttrot.com/v1/"

In [20]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def write_to_file(file_path, content):
    with open(file_path, "w") as f:
        f.write(content)
        
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
result_path = os.path.join("gpt4-count-results")
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [18]:
with open('LEVIR-MCI-dataset/LevirCCcaptions-v2-f.json', 'r') as f:
    datajson = json.load(f)

datajson = [x for x in datajson if x['filepath'] == 'test']

In [28]:
def chat_completions3(row):
    client = OpenAI(api_key=API_SECRET_KEY, base_url=BASE_URL)    
    prompt = (
        "Please help determine how many buildings and roads have changed in these two remote sensing images. Return the number of changes in JSON format only, without any additional content. The JSON keys should be 'buildings' and 'roads'."
    )

    images = [f"LEVIR-MCI-dataset/images/test/A/{row['filename']}", f"LEVIR-MCI-dataset/images/test/B/{row['filename']}"]
    base64_images = [encode_image(image) for image in images]

    user_content = [{"type": "text", "text": prompt}]
    base64_images_content = [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high",
            },
        }
        for base64_image in base64_images
    ]
    user_content.extend(base64_images_content)
    messages_template = [{"role": "user", "content": user_content}]

    # 构造请求并调用 API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_template,
        max_tokens=1600,
        temperature=0,
        n=1,
        seed=2024,
    )

    # 打印生成结果
    result = response.choices[0].message.content
    return result

def get_result_count(row):
    file_path = os.path.join(result_path, row['filename'].replace('.png','.txt'))
    if os.path.exists(file_path):
        return
    ans = chat_completions3(row)
    json_pattern = r'\{.*?\}'
    
    match = re.search(json_pattern, ans, re.DOTALL)
    json_str = match.group()
    try:
        # 将提取的JSON字符串转换为字典
        json.loads(json_str)
    except:
        json_str = ans

    write_to_file(file_path, json_str)

In [29]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 并行执行，提高效率
    executor.map(lambda x: get_result_count(x), datajson)

## combine results

In [40]:
global_id = 100001
rslt = []
def get_ans(filename):
    with open(os.path.join("gpt4-count-results", filename.replace(".png", ".txt")), "r") as f:
        return json.load(f)

for row in datajson:
    rslt.append({
        "question_id": global_id,
        "image_id": [f"test/A/{row['filename']}", f"test/B/{row['filename']}"],
        "answer": get_ans(row['filename'])
    })
    global_id += 1
with open('gpt4-count-results.jsonl', 'w') as file:
    for item in rslt:
        file.write(json.dumps(item) + '\n')